# 割当問題

$$
\mathrm{minimize}_{\pi: V\to V} \sum_{i\in V} c_{i, \pi(i)}
$$

In [1]:
import numpy as np
from scipy.optimize import linear_sum_assignment

n = 1000
cost = np.random.randint(100, 1000, size=(n, n))

In [2]:
%%time
row_ind, col_ind = linear_sum_assignment(cost)
print(cost[row_ind, col_ind].sum())

101033
CPU times: user 25.1 ms, sys: 2.35 ms, total: 27.4 ms
Wall time: 26.9 ms


In [3]:
import networkx as nx

G = nx.DiGraph()
#n = len(cost)
for i in range(n):
    G.add_node(i, demand=-1)
    G.add_node(n + i, demand=1)
G.add_weighted_edges_from([(i, n + j, cost[i, j]) for i in range(n) for j in range(n)])

In [4]:
%%time
val, flow = nx.algorithms.flow.network_simplex(G)
print(val)

101033
CPU times: user 10.6 s, sys: 319 ms, total: 10.9 s
Wall time: 11.2 s


In [5]:
%%time
val, flowDict = nx.capacity_scaling(G)
print(val)

101033
CPU times: user 1min 15s, sys: 16.2 s, total: 1min 31s
Wall time: 1min 39s


In [6]:
from pyscipopt import Model, quicksum

V = list(range(n))
model = Model("ap")
x = {}
for i in V:
    for j in V:
        x[i, j] = model.addVar(name=f"x[{i},{j}]")

for j in V:
    model.addCons(quicksum(x[i, j] for i in V) == 1)
for i in V:
    model.addCons(quicksum(x[i, j] for j in V) == 1)

model.setObjective(quicksum(cost[i, j] * x[i, j] for i in V for j in V), sense='minimize')

#model.optimize()
#print(model.getObjVal())

In [7]:
%%time

model.optimize()
print(model.getObjVal())

presolving:
101033.0
CPU times: user 18.3 s, sys: 1.99 s, total: 20.3 s
Wall time: 21.2 s
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 1000000 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (4.6s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (5.1s) no symmetry present (symcode time: 0.08)
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 1000000 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 1000000 variables (0 bin, 0 int, 0 impl, 1000000 cont) and 2000 constraints
   2000 constraints of type <linear>
Presolving Time: 3.73

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
*20.9s|     1 |     0 |  3203 |     - |    LP  |   0 |1000k|2000 |2000 |   0 |  0 |   0 |   0